<a href="https://colab.research.google.com/github/igcortesg/IOA/blob/main/OPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Informacion del Problema

El problema de Ubicación de Instalaciones con Capacitación (CFLP por sus siglas en inglés) es un modelo matemático utilizado para determinar el mejor lugar para abrir nuevas instalaciones o expandir existentes, teniendo en cuenta restricciones como costos de construcción, capacidades limitadas y demanda. Vamos a explorar el problema Single Source CFLP y su solución óptima, así como cómo se extiende al caso Multi-Source CFLP.
Problema Single Source CFLP

El problema Single Source CFLP tiene las siguientes características:

    Un solo origen de suministro
    Una serie de clientes con demandas conocidas
    Un conjunto de posibles ubicaciones para instalar facilidades
    Costos de construir instalaciones
    Capacidades limitadas para cada instalación
    Demanda total que debe ser atendida

La función objetivo generalmente busca minimizar los costos totales de construir instalaciones y transportar productos a los clientes.
Solución óptima del Single Source CFLP

Para encontrar la solución óptima del Single Source CFLP, se utilizan técnicas de programación lineal entera. Algunas estrategias comunes incluyen:

    Formulación matemática precisa del problema
    Uso de métodos heurísticos para problemas grandes
    Aplicación de algoritmos exactos para problemas más pequeños

La solución óptima garantiza que:

    Se cumplan todas las restricciones (capacidad, demanda, etc.)
    Se minimice el costo total
    Se maximice la eficiencia en la asignación de clientes a instalaciones

Extensión al Multi-Source CFLP

El Multi-Source CFLP es una versión extendida del problema que incorpora múltiples fuentes de suministro. Las principales diferencias son:

    Múltiples orígenes de suministro con diferentes precios de producción
    Posibilidad de utilizar múltiples fuentes para atender la misma demanda
    Consideración adicional de costos de producción y transporte desde diferentes fuentes

Solución óptima del Multi-Source CFLP

Para resolver el Multi-Source CFLP, se requieren técnicas más avanzadas debido a la complejidad añadida:

    Formulación matemática más elaborada que incluye múltiples fuentes de suministro
    Uso de métodos aproximados como aproximaciones lineales o programación cuádratica
    Técnicas de programación entera mixta para manejar múltiples decisiones binarias y contínuas

La solución óptima del Multi-Source CFLP debe considerar:

    La elección óptima entre diferentes fuentes de suministro para cada cliente
    La asignación eficiente de clientes a instalaciones
    El equilibrio entre costos de producción, transporte y construcción

En resumen, mientras que el Single Source CFLP es un problema bien definido, el Multi-Source CFLP introduce una dimensión adicional de complejidad que requiere técnicas más sofisticadas para encontrar la solución óptima. Ambos problemas son fundamentales en la teoría de la ubicación y tienen aplicaciones prácticas importantes en diversos campos industriales y logísticos.

# Importar Carpeta

In [ ]:
%pip install -q amplpy

In [ ]:
import pandas as pd
from google.colab import userdata
import random

In [ ]:
from amplpy import AMPL, ampl_notebook
ampl = ampl_notebook(
    modules="gurobi", # modules to install
    license_uuid='bb2c71f6-2b4c-4570-8eaf-4db0b0d7349a') # license to use

Licensed to Bundle #6733.7184 expiring 20241231: ICI5142-01 INVESTIGACION DE OPERACIONES AVANZADAS (Advanced Operations Research), Prof. Guillermo Cabrera-Guerrero, Pontificia Universidad Catolica de Valparaiso.


# Funciones Importar Archivos


In [ ]:
# Funcion para importar un archivo de texto tipo capXX.txt


def importarCap(ruta):
    with open(ruta, 'r') as file:
        content = file.read().split()
    tiendas = int(content[0])
    clientes = int(content[1])

    oferta = []
    precioF = []
    for i in range(2, tiendas*2+2, 2):
        oferta.append(int(content[i]))
        precioF.append(float(content[i+1]))

    # inicio represent where it start the matrix of clients
    inicio = (tiendas*2)+2
    demanda = []
    costoV = []

    for i in range(inicio, inicio+(tiendas+1)*clientes, tiendas+1):
        demanda.append(int(content[i]))
        fila = []
        for j in range(i+1, i+tiendas+1):
            fila.append(float(content[j]))
        costoV.append(fila)

    return tiendas, clientes, oferta, precioF, demanda, costoV


In [ ]:
# Funcion para importar un archivo de texto tipo numxnum.txt
def importarNumxNum(ruta):
    with open(ruta, 'r') as file:
        lines = file.readlines()

    # Extract the number of shops and customers from the first line
    tiendas, clientes = map(int, lines[0].split())

    # Find the separators and parse each section
    section_indices = [i for i, line in enumerate(
        lines) if line.strip() == '*']

    # Parse shop capacities and fixed costs
    oferta = []
    precioF = []
    for line in lines[section_indices[0] + 1:section_indices[1]]:
        capacity, fixed_cost = map(float, line.split())
        oferta.append(capacity)
        precioF.append(fixed_cost)

    # Parse customer demands
    demanda = []
    for line in lines[section_indices[1] + 1:section_indices[2]]:
        demanda.append(float(line.strip()))

    # Parse cost matrix
    costoV = []
    for line in lines[section_indices[2] + 1:]:
        fila = list(map(float, line.split()))
        costoV.append(fila)

    # Print parsed results
    #print("cantidad de tiendas \n" + str(tiendas))
    #print("cantidad de clientes \n" + str(clientes))
    #print("oferta\n" + str(oferta))
    #print("precio fijo\n" + str(precioF))
    #print("demanda\n" + str(demanda))
    #print("Costo Variable")
    printMatrix(costoV)

    return tiendas, clientes, oferta, precioF, demanda, costoV

In [ ]:
# funcion para importar (elije que funcion ocupar)


def importarArchivo(ruta):
    if "cap" in ruta:
        return importarCap(ruta)
    if "x" in ruta:
        return importarNumxNum(ruta)
    else:
        try:
            raise Exception("No se reconoce el archivo")
        except Exception as e:
            print(e)

In [ ]:
# Funcion para importar un archivo de texto tipo capa.txt
def importarCapa(ruta):
    try:
        with open(ruta, 'r') as file:
            content = file.read().split()

        # Leer la cantidad de tiendas y clientes
        tiendas = int(content[0])
        clientes = int(content[1])

        print(f"Cantidad de tiendas: {tiendas}")
        print(f"Cantidad de clientes: {clientes}")

        # Leer la capacidad de cada tienda y su costo fijo
        oferta = []
        precioF = []
        for i in range(2, tiendas * 2 + 2, 2):
            oferta.append(int(content[i]))
            precioF.append(float(content[i + 1]))

        # Mostrar capacidades y costos fijos
        print("Capacidades de oferta:", oferta)
        print("Costos fijos:", precioF)

        # Iniciar donde empieza la matriz de clientes
        inicio = tiendas * 2 + 2

        # Leer demandas y costos variables
        demanda = []
        costoV = []
        for i in range(inicio, inicio + (clientes * (tiendas + 1)), tiendas + 1):
            demanda.append(int(content[i]))
            fila = [float(content[j]) for j in range(i + 1, i + tiendas + 1)]
            costoV.append(fila)

        # Mostrar demanda y matriz de costos variables
        print("Demanda de clientes:", demanda)
        print("Matriz de costos variables:")
        for fila in costoV:
            print(fila)

        # Retornar las variables
        return tiendas, clientes, oferta, precioF, demanda, costoV

    except Exception as e:
        print(f"Error al leer el archivo: {e}")



# Funciones Auxiliares


## Funciones Importantes

In [ ]:
def formatoCostoV(costoV,clientes,tiendas):
  costoV_dict = {}
  for cliente in range(clientes):
      for tienda in range(tiendas):
          costoV_dict[(cliente+1, tienda+1)] = costoV[cliente][tienda]
  return costoV_dict

In [ ]:
# funcion para acotar espacion de AMPL


def acotarEspacio(tiendas, clientes, oferta, precioF, demanda, costoV, tiendasAbiertas):
    tiendasR = 0
    clientesR = clientes
    ofertaR = []
    precioFR = []
    demandaR = demanda
    # Cuenta tiendas abiertas
    for i in range(tiendas):
      if tiendasAbiertas[i] == 1:
            tiendasR += 1
    #genera costoVR como una matris de Cliente X TiendasAbiertas(solo la que tienen 1)
    costoVR = [[0 for _ in range(tiendasR)] for _ in range(clientes)]
    # genera tienda abiertasR rellena de 1
    tiendasAbiertasR = [1 for _ in range(tiendasR)]

    cont=0
    for i in range(tiendas):
        if tiendasAbiertas[i] == 1:
            ofertaR.append(oferta[i])
            precioFR.append(precioF[i])
            for j in range(clientes):
              costoVR[j][cont]=costoV[j][i]
            cont+=1

    return (tiendasR, clientesR, ofertaR, precioFR, demandaR, costoVR, tiendasAbiertasR)

In [ ]:
def evaluar(tiendas, clientes, oferta, precioF, demanda, costoV, tiendasAbiertas, tipo):
    t, c, o, p, d, cV, ta = acotarEspacio(tiendas, clientes, oferta, precioF, demanda, costoV, tiendasAbiertas)
    if tipo == 1:
        return CFLP_1S(t, c, o, p, d, cV, ta)
    if tipo == 2:
        return CFLP_MS(t, c, o, p, d, cV, ta)

## Funciones Weonas

In [ ]:
def largoNumero(element, width=10):
    return f"{element:.4f}".ljust(width)


In [ ]:
# Mostrar matriz ordenada
def printMatrix(matrix):

    for fila in matrix:
        for valor in fila:
            print(largoNumero(valor), end="\t")
        print()


In [ ]:
# Funcion para mostrar datos
def printData(tiendas, clientes, oferta, precioF, demanda, costoV, tiendasAbiertas):
    print("Cantidad de tiendas: " + str(tiendas))
    print("Cantidad de clientes: " + str(clientes))
    print("Oferta: " + str(oferta))
    print("Precio fijo: " + str(precioF))
    print("Demanda: " + str(demanda))
    print("Costo Variable")
    printMatrix(costoV)
    print("Tiendas abiertas: " + str(tiendasAbiertas))

In [ ]:
def tirarMoneda():
    return random.choice([0, 1])


# Funciones AMPL

In [ ]:
# CFLP single-source 100% AMPL


def CFLP_1S_AMPL(tiendas, clientes, oferta, precioF, demanda, costoV):
    ampl.reset()
    ampl.eval("""
    # Sets
    set Tiendas;
    set Clientes;

    # Variables
    var tiendaAbierta{Tiendas} binary;
    var demandaAsociada{Clientes, Tiendas} binary;  # New binary variable to indicate assignment

    # Parameters
    param oferta{Tiendas};
    param precioF{Tiendas};
    param demanda{Clientes};
    param costoV{Clientes, Tiendas};

    # Objective Function
    minimize CostoTotal:
        sum{t in Tiendas} precioF[t] * tiendaAbierta[t] +
        sum{c in Clientes, t in Tiendas} costoV[c, t] * demandaAsociada[c, t];

    # Constraints
    subject to satisfacerDemanda{c in Clientes}:
        sum{t in Tiendas} demandaAsociada[c, t] = 1;  # Each customer is assigned to exactly one shop

    subject to ofertaDemanda{t in Tiendas}:
        sum{c in Clientes} demanda[c] * demandaAsociada[c, t] <= oferta[t] * tiendaAbierta[t];
            """)

    # Asignar parámetros a AMPL
    ampl.set['Tiendas'] = range(1, tiendas+1)
    ampl.set['Clientes'] = range(1, clientes+1)
    ampl.param['oferta'] = {i+1: oferta[i] for i in range(tiendas)}
    ampl.param['precioF'] = {i+1: precioF[i] for i in range(tiendas)}
    ampl.param['demanda'] = {i+1: demanda[i] for i in range(clientes)}
    ampl.param['costoV'] = costoV

    # Especificar el solver
    ampl.option['solver'] = 'gurobi'

    # Resolver el problema
    ampl.solve()

    # Mostrar la solución
    ampl.display('tiendaAbierta')
    #ampl.display('demandaAsociada')
    ampl.display('CostoTotal')



In [ ]:
# CFLP multy-source 100% AMPL
def CFLP_MS_AMPL(tiendas, clientes, oferta, precioF, demanda, costoV):
    # Crear el modelo AMPL Para MultiSource Source CFLP
    ampl.reset()
    ampl.eval("""
    # Sets
    set Tiendas;
    set Clientes;

    # Variables
    var tiendaAbierta{Tiendas} binary;
    var demandaAsociada{Clientes, Tiendas} >= 0, <= 1;

    # Parameters
    param oferta{Tiendas};
    param precioF{Tiendas};
    param demanda{Clientes};
    param costoV{Clientes, Tiendas};

    # Objective Function
    minimize CostoTotal:
        sum{t in Tiendas} precioF[t] * tiendaAbierta[t] +
        sum{c in Clientes, t in Tiendas} costoV[c, t] * demandaAsociada[c, t];

    # Constraints
    subject to satisfacerDemanda{c in Clientes}:
        sum{t in Tiendas} demandaAsociada[c, t] = 1;  # Each customer is assigned to exactly one shop

    subject to ofertaDemanda{t in Tiendas}:
        sum{c in Clientes} demanda[c] * demandaAsociada[c, t] <= oferta[t] * tiendaAbierta[t];
            """)

    # Asignar parámetros a AMPL
    ampl.set['Tiendas'] = range(1, tiendas+1)
    ampl.set['Clientes'] = range(1, clientes+1)
    ampl.param['oferta'] = {i+1: oferta[i] for i in range(tiendas)}
    ampl.param['precioF'] = {i+1: precioF[i] for i in range(tiendas)}
    ampl.param['demanda'] = {i+1: demanda[i] for i in range(clientes)}
    ampl.param['costoV'] = costoV  # Asignar el diccionario de costoV

    # Especificar el solver
    ampl.option['solver'] = 'gurobi'

    # Resolver el problema
    ampl.solve()

    # Mostrar la solución
    ampl.display('tiendaAbierta')
    ampl.display('demandaAsociada')
    ampl.display('CostoTotal')
    ampl.display('costoV')


In [ ]:
def CFLP_1S(tiendas, clientes, oferta, precioF, demanda, costoV, tiendasAbiertas):
    ampl.reset()
    ampl.eval("""
    # Sets
    set Tiendas;
    set Clientes;

    # Variables
    var demandaAsociada{Clientes, Tiendas} binary;

    # Parameters
    param oferta{Tiendas};
    param precioF{Tiendas};
    param demanda{Clientes};
    param costoV{Clientes, Tiendas};
    param tiendaAbierta{Tiendas} binary;

    # Objective Function
    minimize CostoTotal:
        sum{t in Tiendas} precioF[t] * tiendaAbierta[t] +
        sum{c in Clientes, t in Tiendas} costoV[c, t] * demandaAsociada[c, t];

    # Constraints
    subject to satisfacerDemanda{c in Clientes}:
        sum{t in Tiendas} demandaAsociada[c, t] = 1;  # Each customer is fully served

    subject to ofertaDemanda{t in Tiendas}:
        sum{c in Clientes} demanda[c] * demandaAsociada[c, t] <= oferta[t] * tiendaAbierta[t];

    subject to asignarSiAbierta{c in Clientes, t in Tiendas}:
        demandaAsociada[c, t] <= tiendaAbierta[t];
    """)

    # Assign parameters to AMPL
    ampl.set['Tiendas'] = range(1, tiendas+1)
    ampl.set['Clientes'] = range(1, clientes+1)

    # Create a dictionary for `tiendaAbierta` to indicate open shops
    ampl.param['tiendaAbierta'] = {t: tiendasAbiertas[t-1] for t in range(1, tiendas+1)}
    ampl.param['oferta'] = {t: oferta[t-1] for t in range(1, tiendas+1)}
    ampl.param['precioF'] = {t: precioF[t-1] for t in range(1, tiendas+1)}
    ampl.param['demanda'] = {c: demanda[c-1] for c in range(1, clientes+1)}
    ampl.param['costoV'] = {(c+1, t+1): costoV[c][t] for c in range(clientes) for t in range(tiendas)}

    # Specify the solver
    ampl.option['solver'] = 'gurobi'

    # Solve the problem
    ampl.solve()

    # Display the solution
    #ampl.display('tiendaAbierta')
    #ampl.display('demandaAsociada')
    #ampl.display('CostoTotal')

    # Aviso solucion inviable
    warnings = ampl.get_output("solve;")
    infeasible_constraints = []

    # returns .1 if there is no viable solution
    for line in warnings.split('\n'):
        if 'WARNING:' in line.upper():
            print("AMPL Warning:", line.strip())
            return -1
        if 'INFEASIBLE' in line.upper() or 'NO SOLUTION' in line.upper():
            print("AMPL Error:", line.strip())
            return -1
        if 'all variables eliminated' in line.lower() and '> 0' in line:
            infeasible_constraints.append(line.strip())
            return -1




    return ampl.getValue('CostoTotal')




In [ ]:
 #CFLP multi-source tienda abiertas como parametro


def CFLP_MS(tiendas, clientes, oferta, precioF, demanda, costoV, tiendasAbiertas):
    ampl.reset()
    ampl.eval("""
    # Sets
    set Tiendas;
    set Clientes;

    # Variables
    var demandaAsociada{Clientes, Tiendas} >= 0, <= 1;

    # Parameters
    param oferta{Tiendas};
    param precioF{Tiendas};
    param demanda{Clientes};
    param costoV{Clientes, Tiendas};
    param tiendaAbierta{Tiendas} binary;

    # Objective Function
    minimize CostoTotal:
        sum{t in Tiendas} precioF[t] * tiendaAbierta[t] +
        sum{c in Clientes, t in Tiendas} costoV[c, t] * demandaAsociada[c, t];

    # Constraints
    subject to satisfacerDemanda{c in Clientes}:
        sum{t in Tiendas} demandaAsociada[c, t] = 1;  # Each customer is fully served

    subject to ofertaDemanda{t in Tiendas}:
        sum{c in Clientes} demanda[c] * demandaAsociada[c, t] <= oferta[t] * tiendaAbierta[t];

    subject to asignarSiAbierta{c in Clientes, t in Tiendas}:
        demandaAsociada[c, t] <= tiendaAbierta[t];
    """)

    # Assign parameters to AMPL
    ampl.set['Tiendas'] = range(1, tiendas+1)
    ampl.set['Clientes'] = range(1, clientes+1)

    # Create a dictionary for `tiendaAbierta` to indicate open shops
    ampl.param['tiendaAbierta'] = {t: tiendasAbiertas[t-1] for t in range(1, tiendas+1)}
    ampl.param['oferta'] = {t: oferta[t-1] for t in range(1, tiendas+1)}
    ampl.param['precioF'] = {t: precioF[t-1] for t in range(1, tiendas+1)}
    ampl.param['demanda'] = {c: demanda[c-1] for c in range(1, clientes+1)}
    ampl.param['costoV'] = {(c+1, t+1): costoV[c][t] for c in range(clientes) for t in range(tiendas)}

    # Specify the solver
    ampl.option['solver'] = 'gurobi'

    # Solve the problem
    ampl.solve()

    # Display the solution
    #ampl.display('tiendaAbierta')
    #ampl.display('demandaAsociada')
    #ampl.display('CostoTotal')
    #ampl.display('oferta')
    #ampl.display('demanda')
    #ampl.display('costoV')

    # Aviso solucion inviable
    warnings = ampl.get_output("solve;")
    infeasible_constraints = []
    # returns .1 if there is no viable solution
    for line in warnings.split('\n'):
        if 'WARNING:' in line.upper():
            print("AMPL Warning:", line.strip())
            return -1
        if 'INFEASIBLE' in line.upper() or 'NO SOLUTION' in line.upper():
            print("AMPL Error:", line.strip())
            return -1
        if 'all variables eliminated' in line.lower() and '> 0' in line:
            infeasible_constraints.append(line.strip())
            return -1

    return ampl.getValue('CostoTotal')


# Tabu search


In [ ]:
def tabuSearch(tiendas, clientes, oferta, precioF, demanda, costoV, tipo, maxIter):
    # solucion inicial todas las tiendas abiertas
    tiendasAbiertas = [1]*tiendas
    demandaTotal = sum(demanda)
    if evaluar(tiendas, clientes, oferta, precioF, demanda, costoV, tiendasAbiertas, tipo) == -1:
        return " No hay solucion", " No se encontro solucion"

    costo = evaluar(tiendas, clientes, oferta, precioF, demanda, costoV, tiendasAbiertas, tipo)
    mejorCosto = costo
    mejorSolucion = tiendasAbiertas.copy()

    actualCosto = costo
    actualSolucion = tiendasAbiertas.copy()

    # Memoria a corto y largo plazo
    tabuShort = [0]*tiendas
    tabuLong = [0]*tiendas
    # Lista de tabu ayuda a saber que elementos estan en la lista tabu
    tabuList = []
    for i in range(maxIter):
        mejorVecino=None
        costoVecino=float('inf')
        indiceVecino=0
        for j in range(tiendas):
            if tabuShort[j] == 0:
                tiendasAbiertas[j] = 1 - tiendasAbiertas[j]
                costo = evaluar(tiendas, clientes, oferta, precioF, demanda, costoV, tiendasAbiertas, tipo)
                if costo!=-1:
                  if costo < costoVecino:
                      costoVecino = costo
                      mejorVecino = tiendasAbiertas.copy()
                      indiceVecino=j
                tiendasAbiertas[j] = 1 - tiendasAbiertas[j]
        if costoVecino<mejorCosto:
          mejorCosto=costoVecino
          mejorSolucion=mejorVecino.copy()

        if mejorVecino==None:
          i=0
          while tabuShort[i]>0:
            i+=1
            if i>=tiendas:
              return mejorCosto, mejorSolucion

          tiendasAbiertas[i] = 1 - tiendasAbiertas[i]
          mejorVecino=tiendasAbiertas.copy()
          costoVecino = evaluar(tiendas, clientes, oferta, precioF, demanda, costoV, tiendasAbiertas, tipo)
          tabuLong[i] += 1
          tabuShort[i] = 3*tabuLong[j]
          tabuList.append(i)


        tiendasAbiertas=mejorVecino.copy()
        tabuLong[indiceVecino] += 1
        tabuShort[indiceVecino] = 3*tabuLong[indiceVecino]
        tabuList.append(indiceVecino)
# Hay que arreglar esto
        # if mejorSolucion == tiendasAbiertas:
        #   i = random.randint(0, tiendas-1)
        #   tiendasAbiertas[i] = 1 - tiendasAbiertas
        #   mejorCosto = evaluar(tiendas, clientes, oferta, precioF, demanda, costoV, tiendasAbiertas, 2)
        #   mejorSolucion = tiendasAbiertas.copy()
        #   tabuLong[i] += 1
        #   tabuShort[i] = 3*tabuLong[j]
        #   tabuList.append(i)

        for j in tabuList:
            tabuShort[j] -= 1
            if tabuShort[j] == 0:
                tabuList.remove(j)
    print(tabuShort)
    print(tabuLong)
    return mejorCosto, mejorSolucion

In [ ]:
def ofertaTotal(oferta, tiendasAbiertas):
    ofertaTotal = 0
    for i in range(len(oferta)):
        ofertaTotal += oferta[i]*tiendasAbiertas[i]
    return ofertaTotal

In [ ]:
def chavoDel8(tienda, cliente, oferta, precioF, demanda, costoV, tiendasAbiertas):
    # Selecciona mejor vecino
    mejorCosto = float('inf')
    mejorSolucion = None
    for i in range(tienda):
        tiendasAbiertas[i] = 1 - tiendasAbiertas
        costo = evaluar(tienda, cliente, oferta, precioF, demanda, costoV, tiendasAbiertas, 2)
        if costo!=-1:
          if costo < mejorCosto:
              mejorCosto = costo
              mejorSolucion = tiendasAbiertas.copy()
        tiendasAbiertas[i] = 1 - tiendasAbiertas
    if mejorSolucion == tiendasAbiertas:
        i = random.randint(0, tienda-1)
        tiendasAbiertas[i] = 1 - tiendasAbiertas
        mejorCosto = evaluar(tienda, cliente, oferta, precioF, demanda, costoV, tiendasAbiertas, 2)
        mejorSolucion = tiendasAbiertas.copy()

    return mejorCosto, mejorSolucion

# Main

In [ ]:
from typing import Type
ruta = r"/content/capc(5000).txt"
try:
    tiendas, clientes, oferta, precioF, demanda, costoV = importarArchivo(ruta)
except Exception as e:
    print(f"Error al importar el archivo: {str(e)}")
#tiendasAbiertas=[0]*tiendas
#costoAux=formatoCostoV(costoV,clientes,tiendas)
#CFLP_1S_AMPL(tiendas, clientes, oferta, precioF, demanda, costoAux)
#print(CFLP_MS_AMPL(tiendas, clientes, oferta, precioF, demanda, costoV))
#ite=tiendas+clientes
mC,mS=tabuSearch(tiendas, clientes, oferta, precioF, demanda, costoV, 2, 100)
print("Mejor Costo\n"+str(mC))
print("Mejor Solucion\n"+str(mS))

Gurobi 12.0.0: Gurobi 12.0.0: optimal solution; objective 55941498.43
0 simplex iterations
Gurobi 12.0.0: Gurobi 12.0.0: optimal solution; objective 55941498.43
0 simplex iterations
Gurobi 12.0.0: Gurobi 12.0.0: optimal solution; objective 55330142.22
0 simplex iterations
Gurobi 12.0.0: Gurobi 12.0.0: optimal solution; objective 55513857.84
0 simplex iterations
Gurobi 12.0.0: Gurobi 12.0.0: optimal solution; objective 55297607.97
0 simplex iterations
Gurobi 12.0.0: Gurobi 12.0.0: optimal solution; objective 55343949.48
0 simplex iterations
Gurobi 12.0.0: Gurobi 12.0.0: optimal solution; objective 55374734.23
0 simplex iterations
Gurobi 12.0.0: Gurobi 12.0.0: optimal solution; objective 55541223.66
0 simplex iterations
Gurobi 12.0.0: Gurobi 12.0.0: optimal solution; objective 55446089.15
0 simplex iterations
Gurobi 12.0.0: Gurobi 12.0.0: o